# Demo of the Models on Function Integration

In [1]:
import os

import numpy
import pandas
import sympy
import torch

from src.envs import build_env
from src.envs.char_sp import InvalidPrefixExpression
from src.envs.sympy_utils import simplify
from src.model import build_modules
from src.utils import AttrDict
from src.utils import to_cuda

Use the Model to Find the Integral of a Function: three approaches to cover all the problem space.

## Forward Approach (FWD)

Start from a function $f$, compute its integral $F = \int f$, and try to recover $F$ from $f$.

**Pros**:
- Representative sample of the subset of the problem space that can be solved by an external symbolic mathematical
  framework.

**Cons**:
- Use only function that can solve by symbolic mathematical framework.
- Time expensive.

### Build Environment - Reload Model

Get Trained Model:

In [2]:
model_path = '../models/integrations/fwd.pth'
assert os.path.isfile(model_path)

Set the Parameters for Environment and for the Model:

 Environment:
- **env_name**: SymPy character environment.
- **int_base**: integer representation base.
- **balanced**: balanced representation (base > 0).
- **positive**: do not sample negative numbers.
- **precision**: float numbers precision.
- **n_variables**: number of variables in expressions (between 1 and 4).
- **n_coefficients**: number of coefficients in expressions (between 0 and 10).
- **leaf_probs**: leaf probabilities of being a variable, a coefficient, an integer, or a constant.
- **max_len**: maximum sequences length.
- **max_int**: maximum integer value.
- **max_ops**: maximum number of operators.
- **max_ops_G**: maximum number of operators for G in IBP.
- **clean_prefix_expr**: clean prefix expressions (f x -> Y, derivative f x x -> Y').
- **rewrite_functions**: rewrite expressions with a given SymPy function.
- **tasks**: tasks to run (prim_fwd, prim_bwd, prim_ibp, ode1, ode2).
- **operators**: considered operators (add, sub, mul, div), followed by (unnormalized) sampling probabilities.


 Model:
- **cpu**: run on CPU.
- **emb_dim**: embedding layer size.
- **n_enc_layers**: number of transformer layers in the encoder.
- **n_dec_layers**: number of transformer layers in the decoder.
- **n_heads**: number of transformer heads.
- **dropout**: dropout.
- **attention_dropout**: dropout in the attention layer.
- **sinusoidal_embeddings**: use sinusoidal embeddings.
- **share_inout_emb**: share input and output embeddings.
- **reload_model**: reload a pretrained model.

In [3]:
params = AttrDict({

    # Environment Parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'prim_fwd',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,'
                 'acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # Model Parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': model_path,

})

Set the Environment with SymPy:

In [4]:
env = build_env(params)
x = env.local_dict['x']

The primary components of the model are one encoder and one decoder network. The encoder turns each item into a
corresponding hidden vector containing the item and its context. The decoder reverses the process, turning the vector
into an output item, using the previous output as the input context.

Build Model Modules:

In [5]:
modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

### Declare input function $f$ and Compute Integral Function $F$

Declare $f$, the input function for the Model:

In [6]:
# f_infix = '(sqrt((x**(-1))*(x+((-1)*(x**2)))))'
# f_infix = '((sqrt(1+((-1)*(x**2))))+(2*(x**2)))'
# f_infix = 'cos(x)*exp(2*x+1)'
# f_infix = '2/(1-4*x)'
# f_infix = '-(1-2*x)/(x*(1-4*x))'
# f_infix = '-2/(1-4*x) + 1/x'
# f_infix = '1/(1-4*x)**(3/2)'
# f_infix = '(x**2*(3-x))/(6*(1-x))'
f_infix = '(((-1)*x)+(2*((x+(tan(atanh(2))))**(-1))))'

Converts **f_infix** to a type that can be used inside SymPy:

In [7]:
f = sympy.sympify(f_infix, locals=env.local_dict)
f

-x + 2/(x + tan(atanh(2)))

Compute $F$, or $\int f$, the integral function the model has to predict:

In [8]:
F = f.integrate(x, risch=True)
F = F.doit()
F

-x**2/2 + 2*log(x + tan(atanh(2)))

### Compute Prefix Representations

In [9]:
F_prefix = env.sympy_to_prefix(F)
f_prefix = env.sympy_to_prefix(f)
print(f"F with Prefix Notation:\n{F_prefix}\n")
print(f"f with Prefix Notation:\n{f_prefix}")

F with Prefix Notation:
['add', 'mul', 'INT+', '2', 'ln', 'add', 'x', 'tan', 'atanh', 'INT+', '2', 'mul', 'div', 'INT-', '1', 'INT+', '2', 'pow', 'x', 'INT+', '2']

f with Prefix Notation:
['add', 'mul', 'INT-', '1', 'x', 'mul', 'INT+', '2', 'pow', 'add', 'x', 'tan', 'atanh', 'INT+', '2', 'INT-', '1']


### Encode Input

Clean prefix expressions before they are converted to PyTorch data.

Examples:
- f x  -> Y
- derivative f x x  -> Y'

In [10]:
x1_prefix = env.clean_prefix(['sub', 'derivative', 'f', 'x', 'x'] + f_prefix)
print(f"f Clean Prefix Notation:\n{x1_prefix}")

f Clean Prefix Notation:
['sub', "Y'", 'add', 'mul', 'INT-', '1', 'x', 'mul', 'INT+', '2', 'pow', 'add', 'x', 'tan', 'atanh', 'INT+', '2', 'INT-', '1']


Create a PyTorch LongTensor for storing $f$ as a sequence of indexes based on prefix clean notation "words" (Word to
index dictionary is defined inside the Model environment):

In [11]:
x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)
x1.transpose(0, 1)

tensor([[ 0, 67, 79, 33, 54, 72, 82, 12, 54, 71, 83, 55, 33, 12, 68, 39, 71, 83,
         72, 82,  0]])

Move PyTorch tensors to CUDA (GPU):

In [12]:
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)

Encodes the “meaning” of the input sequence into a single vector, with the Encoder of the Model:

In [13]:
with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

encoded

tensor([[[ 0.0063,  0.0216, -0.0062,  ..., -0.0070,  0.0088,  0.0114],
         [ 0.0440, -0.2198, -0.0309,  ..., -0.0477, -0.0275,  0.0669],
         [-0.0233,  0.0610, -0.0577,  ...,  0.0312, -0.0690,  0.0296],
         ...,
         [ 0.0066, -0.1779,  0.0603,  ..., -0.0720, -0.0356,  0.1443],
         [-0.1481, -0.1231,  0.0599,  ...,  0.0024, -0.0812,  0.1175],
         [ 0.0025,  0.0126, -0.0751,  ..., -0.0671, -0.0823, -0.0234]]],
       device='cuda:0')

### Decode with Beam Search

Instead of picking a single output, a sequence (in this case an hypothesis of an integral), multiple highly probable
choices are retained.

Declare beam size:

In [14]:
beam_size = 10

Takes the encoder output vector and outputs multiple sequences of "words", that in this case should represent integral
function $F$ hypothesis, using the Decoder of the model.

In [15]:
with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1,
                                       max_len=params.max_len)
assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

### View the Results

Input function $f$:

In [16]:
f

-x + 2/(x + tan(atanh(2)))

Integral function $F$ to find:

In [17]:
F

-x**2/2 + 2*log(x + tan(atanh(2)))

Extract scores and integrals hypotheses:

In [18]:
rows = numpy.arange(1, beam_size + 1)
columns = ['Score', 'Integral Hypothesis', 'Valid']
results = []

for score, sequence in sorted(hypotheses, reverse=True):
    # Parse decoded hypothesis
    ids = sequence[1:].tolist()  # Decoded token IDs
    hyp_prefix = [env.id2word[word_id] for word_id in ids]  # Convert to prefix notation

    try:
        hyp_infix = env.prefix_to_infix(hyp_prefix)  # Convert to infix notation
        hyp_sympy = env.infix_to_sympy(hyp_infix)  # Convert to SymPy

        # Check whether we recover "f" if we differentiate the hypothesis
        # Note that sometimes, SymPy fails to show that hyp' - f == 0, and the result is considered as invalid, although it
        # may be correct
        validation = "YES" if simplify(hyp_sympy.diff(x) - f, seconds=1) == 0 else "NO"

        # Transform hypothesis to a valid latex expression
        hyp_expr = "$" + sympy.latex(env.infix_to_sympy(hyp_infix)) + "$"

    except InvalidPrefixExpression:
        validation = "INVALID PREFIX EXPRESSION"
        hyp_expr = hyp_prefix

    # Prepare results
    results.append([score, hyp_expr, validation])

Print results:

In [19]:
pandas.set_option('max_colwidth', None)
pandas.DataFrame(results, index=rows, columns=columns).style.set_properties(**{'text-align': 'center'})

,Score,Integral Hypothesis,Valid
1,-0.000000,$- \frac{x^{2}}{2} + 2 \log{\left(x + \tan{\left(\operatorname{atanh}{\left(2 \right)} \right)} \right)}$,YES
2,-0.671518,$- \frac{x^{2}}{2} + 2 \log{\left(x + \tan{\left(\operatorname{atanh}{\left(2 \right)} \right)} \right)}$,YES
3,-0.726764,$- \frac{x^{2} \tan{\left(\operatorname{atanh}{\left(2 \right)} \right)}}{2} + 2 \pi \log{\left(x + \tan{\left(\operatorname{atanh}{\left(2 \right)} \right)} \right)}$,NO
4,-0.748399,$- x^{2} + 2 \pi \log{\left(x + \tan{\left(\operatorname{atanh}{\left(2 \right)} \right)} \right)}$,NO
5,-0.788807,$- \frac{x^{2}}{2} + 2 \log{\left(x + \tan{\left(\operatorname{atanh}{\left(x^{2} \right)} \right)} \right)}$,NO
6,-0.851299,$- \frac{x^{2}}{2} + 2 \log{\left(x + \tan{\left(\operatorname{atanh}{\left(5 \right)} \right)} \right)}$,NO
7,-0.880198,$- \frac{x^{2}}{2} + 2 \log{\left(x + 2 \right)}$,NO
8,-0.880670,$- \frac{x^{2}}{2} + 2 \log{\left(x + \tan{\left(\operatorname{atanh}{\left(3 \right)} \right)} \right)}$,NO
9,-8.903655,$0$,NO
10,-32.509010,[],INVALID PREFIX EXPRESSION


## Backward Approach (BWD)

Start from a function $F$, compute its derivative $f = F'$, and try to recover $F$ from $f$.

**Pros**:
- Always possible.
- Extremely fast even for large expressions.
- Does not depend on external symbolic integration system.

**Cons**:
- Very unlikely to generate the integral of simple functions.

### Build Environment - Reload Model

In [20]:
model_path = '../models/integrations/bwd.pth'
assert os.path.isfile(model_path)

params = AttrDict({

    # Environment Parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'prim_bwd',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,'
                 'acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # Model Parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': model_path,

})

env = build_env(params)

modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

### Declare Integral function $F$ and Compute Input Function $f$

Declare $F$, the integral function the model has to predict:

In [21]:
# F_infix = 'x * tan(exp(x)/x)'
# F_infix = 'x * cos(x**2) * tan(x)'
# F_infix = 'cos(x**2 * exp(x * cos(x)))'
F_infix = 'ln(cos(x + exp(x)) * sin(x**2 + 2) * exp(x) / x)'

Converts **F_infix** to a type that can be used inside SymPy:

In [22]:
F = sympy.sympify(F_infix, locals=env.local_dict)
F

log(exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x)

Get $f$, or $F'$, that the model will take as input:

In [23]:
f = F.diff(x)
f

x*(2*exp(x)*cos(x + exp(x))*cos(x**2 + 2) - (exp(x) + 1)*exp(x)*sin(x + exp(x))*sin(x**2 + 2)/x + exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x - exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x**2)*exp(-x)/(sin(x**2 + 2)*cos(x + exp(x)))

### Compute Prefix Representations

In [24]:
F_prefix = env.sympy_to_prefix(F)
f_prefix = env.sympy_to_prefix(f)
print(f"F with Prefix Notation:\n{F_prefix}\n")
print(f"f with Prefix Notation:\n{f_prefix}")

F with Prefix Notation:
['ln', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2']

f with Prefix Notation:
['mul', 'x', 'mul', 'pow', 'cos', 'add', 'x', 'exp', 'x', 'INT-', '1', 'mul', 'pow', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'INT-', '1', 'mul', 'add', 'mul', 'INT+', '2', 'mul', 'cos', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'exp', 'x', 'add', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'add', 'mul', 'INT-', '1', 'mul', 'pow', 'x', 'INT-', '2', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'mul', 'INT-', '1', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'add', 'INT+', '1', 'exp', 'x', 'mul', 'exp', 'x', 'mul', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'sin', 'add', 'x', 'exp',

### Encode Input

In [25]:
x1_prefix = env.clean_prefix(['sub', 'derivative', 'f', 'x', 'x'] + f_prefix)
print(f"f Clean Prefix Notation:\n{x1_prefix}")

x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)

f Clean Prefix Notation:
['sub', "Y'", 'mul', 'x', 'mul', 'pow', 'cos', 'add', 'x', 'exp', 'x', 'INT-', '1', 'mul', 'pow', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'INT-', '1', 'mul', 'add', 'mul', 'INT+', '2', 'mul', 'cos', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'exp', 'x', 'add', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'add', 'mul', 'INT-', '1', 'mul', 'pow', 'x', 'INT-', '2', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'mul', 'INT-', '1', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'add', 'INT+', '1', 'exp', 'x', 'mul', 'exp', 'x', 'mul', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2', 'sin', 'add', 'x', 'exp', 'x', 'exp', 'mul', 'INT-', '1', 'x']


In [26]:
len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)
x1.transpose(0, 1)

tensor([[ 0, 67, 79, 54, 12, 54, 55, 40, 33, 12, 48, 12, 72, 82, 54, 55, 64, 33,
         71, 83, 55, 12, 71, 83, 72, 82, 54, 33, 54, 71, 83, 54, 40, 33, 71, 83,
         55, 12, 71, 83, 54, 40, 33, 12, 48, 12, 48, 12, 33, 54, 55, 12, 72, 82,
         54, 40, 33, 12, 48, 12, 54, 48, 12, 64, 33, 71, 83, 55, 12, 71, 83, 33,
         54, 72, 82, 54, 55, 12, 72, 83, 54, 40, 33, 12, 48, 12, 54, 48, 12, 64,
         33, 71, 83, 55, 12, 71, 83, 54, 72, 82, 54, 55, 12, 72, 82, 54, 33, 71,
         82, 48, 12, 54, 48, 12, 54, 64, 33, 71, 83, 55, 12, 71, 83, 64, 33, 12,
         48, 12, 48, 54, 72, 82, 12,  0]], device='cuda:0')

In [27]:
with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

encoded

tensor([[[-0.0312, -0.0124,  0.0321,  ...,  0.0168, -0.0144,  0.0064],
         [ 0.0435, -0.0574,  0.0247,  ...,  0.0034,  0.0514, -0.0321],
         [ 0.0775,  0.0448, -0.0215,  ...,  0.0198, -0.0047,  0.0607],
         ...,
         [-0.0762,  0.0193,  0.0408,  ...,  0.0788,  0.0954,  0.2066],
         [-0.1294,  0.0265,  0.0799,  ..., -0.1040,  0.1965,  0.1924],
         [-0.0523,  0.0090, -0.0042,  ...,  0.0234,  0.0123, -0.0635]]],
       device='cuda:0')

### Decode with Beam Search

In [28]:
beam_size = 10

with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1,
                                       max_len=params.max_len)
assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

### View the Results

Input function $f$:

In [29]:
f

x*(2*exp(x)*cos(x + exp(x))*cos(x**2 + 2) - (exp(x) + 1)*exp(x)*sin(x + exp(x))*sin(x**2 + 2)/x + exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x - exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x**2)*exp(-x)/(sin(x**2 + 2)*cos(x + exp(x)))

Integral function $F$ to find:

In [30]:
F

log(exp(x)*sin(x**2 + 2)*cos(x + exp(x))/x)

Extract scores and integrals hypotheses:

In [31]:
rows = numpy.arange(1, beam_size + 1)
columns = ['Score', 'Integral Hypothesis', 'Valid']
results = []

for score, sequence in sorted(hypotheses, reverse=True):
    # Parse decoded hypothesis
    ids = sequence[1:].tolist()  # Decoded token IDs
    hyp_prefix = [env.id2word[word_id] for word_id in ids]  # Convert to prefix notation

    try:
        hyp_infix = env.prefix_to_infix(hyp_prefix)  # Convert to infix notation
        hyp_sympy = env.infix_to_sympy(hyp_infix)  # Convert to SymPy

        # Check whether we recover "f" if we differentiate the hypothesis
        # Note that sometimes, SymPy fails to show that hyp' - f == 0, and the result is considered as invalid, although it
        # may be correct
        validation = "YES" if simplify(hyp_sympy.diff(x) - f, seconds=1) == 0 else "NO"

        # Transform hypothesis to a valid latex expression
        hyp_expr = "$" + sympy.latex(env.infix_to_sympy(hyp_infix)) + "$"

    except InvalidPrefixExpression:
        validation = "INVALID PREFIX EXPRESSION"
        hyp_expr = hyp_prefix

    # Prepare results
    results.append([score, hyp_expr, validation])

Print results:

In [32]:
pandas.set_option('max_colwidth', None)
pandas.DataFrame(results, index=rows, columns=columns).style.set_properties(**{'text-align': 'center'})

,Score,Integral Hypothesis,Valid
1,-0.000010,$\log{\left(\frac{e^{x} \sin{\left(x^{2} + 2 \right)} \cos{\left(x + e^{x} \right)}}{x} \right)}$,YES
2,-0.364873,$\operatorname{atan}{\left(\tan{\left(\log{\left(\frac{e^{x} \sin{\left(x^{2} + 2 \right)} \cos{\left(x + e^{x} \right)}}{x} \right)} \right)} \right)}$,YES
3,-0.402405,$\log{\left(\frac{e^{- x} \sin{\left(x^{2} + 2 \right)} \cos{\left(x + e^{x} \right)}}{x} \right)}$,NO
4,-0.411729,$\log{\left(\frac{e^{x} \sin{\left(x \left(x + \frac{2}{x}\right) \right)} \cos{\left(x + e^{x} \right)}}{x} \right)}$,YES
5,-0.421149,$\log{\left(\frac{e^{x} \sin{\left(x \left(x + \frac{1}{x}\right) + 1 \right)} \cos{\left(x + e^{x} \right)}}{x} \right)}$,YES
6,-0.434728,$\log{\left(- \frac{e^{x} \sin{\left(x^{2} + 2 \right)} \cos{\left(x + e^{x} \right)}}{x} \right)}$,YES
7,-0.435394,$\log{\left(\frac{e^{x} \sin{\left(x \left(x - 1\right) + x + 2 \right)} \cos{\left(x + e^{x} \right)}}{x} \right)}$,YES
8,-0.454395,"['exp', 'x', 'ln', 'mul', 'pow', 'x', 'INT-', '1', 'mul', 'cos', 'add', 'x', 'exp', 'x', 'mul', 'exp', 'x', 'sin', 'add', 'INT+', '2', 'pow', 'x', 'INT+', '2']",INVALID PREFIX EXPRESSION
9,-0.512820,$\log{\left(\frac{e^{x} \sin{\left(x^{2} + 2 \right)} \cos{\left(x + e^{x} \right)}}{x} \right)}$,YES
10,-0.869112,$\log{\left(\frac{e^{x} \sin{\left(x \left(x + \frac{1}{x^{2}}\right) + 1 \right)} \cos{\left(x + e^{x} \right)}}{x} \right)}$,NO


## Integration by Parts Approach (IBP)

Start from two function $F$ and $G$, we compute their respective derivatives $f$ and $g$. If we know the integral of
$fG$ then we can compute the integral of $Fg$ ($H$) as:

$$ H = \int Fg = FG − \int fG $$

Use the model to try to recover $\int Fg $ from $Fg$.

**Pros**:
- We can generate the integrals of functions like $x^{10}\sin(x)$ without resorting to an external symbolic integration
  system.

**Cons**:
- Very Time expensive.

### Build Environment - Reload Model

In [33]:
model_path = '../models/integrations/ibp.pth'
assert os.path.isfile(model_path)

params = AttrDict({

    # Environment Parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'prim_ibp',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,'
                 'acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # Model Parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 8,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': model_path,

})

env = build_env(params)

modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

### Declare input function $F$ and $G$, Compute Derivatives $f$ and $g$, Compute Integral Function $H$ with IBP

Declare $F$ and $G$ functions:

In [34]:
F_infix = '(x**3)'
G_infix = 'x^2/2+5*x'


Converts **F_infix** and **G_infix** to types that can be used inside SymPy:

In [35]:
F = sympy.sympify(F_infix, locals=env.local_dict)
F

x**3

In [36]:
G = sympy.sympify(G_infix, locals=env.local_dict)
G

x**2/2 + 5*x

Compute the derivatives $f$ and $g$:

In [37]:
f = F.diff(x)
f

3*x**2

In [38]:
g = G.diff(x)
g

x + 5

Compute $ h = Fg $, the input function for the Model:

In [39]:
h = F * g
h

x**3*(x + 5)

Compute $ H = \int Fg = FG − \int fG $, the integral function the model has to predict:

In [40]:
H = F * G - sympy.integrate(f * G, x, risch=True).doit()
H = simplify(H, seconds=1)
H

x**4*(4*x + 25)/20

### Compute Prefix Representations

In [41]:
H_prefix = env.sympy_to_prefix(H)
h_prefix = env.sympy_to_prefix(h)
print(f"H with Prefix Notation:\n{H_prefix}\n")
print(f"h with Prefix Notation:\n{h_prefix}")

H with Prefix Notation:
['mul', 'div', 'INT+', '1', 'INT+', '2', '0', 'mul', 'pow', 'x', 'INT+', '4', 'add', 'INT+', '2', '5', 'mul', 'INT+', '4', 'x']

h with Prefix Notation:
['mul', 'pow', 'x', 'INT+', '3', 'add', 'INT+', '5', 'x']


### Encode Input

In [42]:
x1_prefix = env.clean_prefix(['sub', 'derivative', 'f', 'x', 'x'] + h_prefix)
print(f"h Clean Prefix Notation:\n{x1_prefix}")

h Clean Prefix Notation:
['sub', "Y'", 'mul', 'pow', 'x', 'INT+', '3', 'add', 'INT+', '5', 'x']


In [43]:
x1 = torch.LongTensor(
    [env.eos_index] +
    [env.word2id[w] for w in x1_prefix] +
    [env.eos_index]
).view(-1, 1)

len1 = torch.LongTensor([len(x1)])
x1, len1 = to_cuda(x1, len1)
x1.transpose(0, 1)

tensor([[ 0, 67, 79, 54, 55, 12, 71, 84, 33, 71, 86, 12,  0]], device='cuda:0')

In [44]:
with torch.no_grad():
    encoded = encoder('fwd', x=x1, lengths=len1, causal=False).transpose(0, 1)

encoded

tensor([[[-0.0063,  0.0075, -0.0431,  ..., -0.0239,  0.0176, -0.0161],
         [ 0.0221,  0.0035,  0.0368,  ..., -0.0141,  0.0493,  0.0671],
         [-0.0368, -0.0014,  0.0096,  ...,  0.0896,  0.0694, -0.0245],
         ...,
         [-0.0509,  0.1097,  0.0223,  ...,  0.1510, -0.0869,  0.0084],
         [-0.1192, -0.0365,  0.0351,  ...,  0.0007,  0.1911, -0.0048],
         [ 0.0079,  0.0603,  0.0750,  ..., -0.0373, -0.0318,  0.0043]]],
       device='cuda:0')

### Decode with Beam Search

In [45]:
beam_size = 10

with torch.no_grad():
    _, _, beam = decoder.generate_beam(encoded, len1, beam_size=beam_size, length_penalty=1.0, early_stopping=1,
                                       max_len=params.max_len)
assert len(beam) == 1
hypotheses = beam[0].hyp
assert len(hypotheses) == beam_size

### View the Results

Input function $h$:

In [46]:
h

x**3*(x + 5)

Integral function $H$ to find:

In [47]:
H

x**4*(4*x + 25)/20

Extract scores and integrals hypotheses:

In [48]:
rows = numpy.arange(1, beam_size + 1)
columns = ['Score', 'Integral Hypothesis', 'Valid']
results = []

for score, sequence in sorted(hypotheses, reverse=True):
    # Parse decoded hypothesis
    ids = sequence[1:].tolist()  # Decoded token IDs
    hyp_prefix = [env.id2word[word_id] for word_id in ids]  # Convert to prefix notation

    try:
        hyp_infix = env.prefix_to_infix(hyp_prefix)  # Convert to infix notation
        hyp_sympy = env.infix_to_sympy(hyp_infix)  # Convert to SymPy

        # Check whether we recover "f" if we differentiate the hypothesis
        # Note that sometimes, SymPy fails to show that hyp' - f == 0, and the result is considered as invalid, although it
        # may be correct
        validation = "YES" if simplify(hyp_sympy.diff(x) - h, seconds=1) == 0 else "NO"

        # Transform hypothesis to a valid latex expression
        hyp_expr = "$" + sympy.latex(env.infix_to_sympy(hyp_infix)) + "$"

    except InvalidPrefixExpression:
        validation = "INVALID PREFIX EXPRESSION"
        hyp_expr = hyp_prefix

    # Prepare results
    results.append([score, hyp_expr, validation])

Print results:

In [49]:
pandas.set_option('max_colwidth', None)
pandas.DataFrame(results, index=rows, columns=columns).style.set_properties(**{'text-align': 'center'})

,Score,Integral Hypothesis,Valid
1,-0.149982,$\frac{x^{4} \left(4 x \sin{\left(1 \right)} + 4 x + 25 \sin{\left(1 \right)} + 25\right)}{5 \left(4 \sin{\left(1 \right)} + 4\right)}$,YES
2,-0.150172,$\frac{x^{4} \left(4 x + 4 x \tan{\left(1 \right)} + 25 + 25 \tan{\left(1 \right)}\right)}{5 \left(4 + 4 \tan{\left(1 \right)}\right)}$,YES
3,-0.150313,$\frac{x^{4} \left(4 x \cos{\left(1 \right)} + 4 x + 25 \cos{\left(1 \right)} + 25\right)}{5 \left(4 \cos{\left(1 \right)} + 4\right)}$,YES
4,-0.165258,$\frac{x^{4} \left(4 x + \log{\left(5^{4 x + 25} \right)} + 25\right)}{5 \left(4 + 4 \log{\left(5 \right)}\right)}$,YES
5,-0.166332,$\frac{x^{4} \left(4 x + \log{\left(2^{4 x + 25} \right)} + 25\right)}{5 \left(4 \log{\left(2 \right)} + 4\right)}$,YES
6,-0.170196,$\frac{x^{4} \left(4 x + \log{\left(2^{8 x + 50} \right)} + 25\right)}{5 \left(4 + 4 \log{\left(4 \right)}\right)}$,YES
7,-0.170329,$\frac{x^{4} \left(500 x + 3125\right)}{2500}$,YES
8,-0.178801,$\frac{x^{4} \left(4 x + 4 x \tan{\left(1 \right)} + 25 + 25 \tan{\left(1 \right)}\right)}{10 \left(2 + 2 \tan{\left(1 \right)}\right)}$,YES
9,-0.179652,$\frac{x^{4} \left(4 x \sin{\left(1 \right)} + 4 x + 25 \sin{\left(1 \right)} + 25\right)}{10 \left(2 \sin{\left(1 \right)} + 2\right)}$,YES
10,-0.182446,$\frac{x^{4} \left(4 x + 4 x e^{5} + 25 + 25 e^{5}\right)}{10 \left(2 + 2 e^{5}\right)}$,YES
